# Introduction

NOTE: You will need a physical network setup to run this code.

`N0`: Control-type. Sends attestation requests to Nodes 1-5.\
`N1`: Sense-type. Generates floating point numbers in fixed ranges and sends them to N2.\
`N2`: Process-type. Processes the received values and generates the corresponding control signal for N3.\
`N3`: Control-type. Uses the received control signal to control output devices (LEDs for simplicity).\
`N4`: Sense-type. Generates floating point numbers in fixed ranges and sends them to N5.\
`N5`: Process and Control-type. Uses the received data to control output devices.

The directed edges in the network creates an "effect" on subsequent nodes in the direction of communication, and each [N0, N1, N2, N3, N4, N5] SRAM sample represents the network state at a given instance.\

In [ ]:
import os
import time
import serial
import random
import pickle
import argparse
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# inport the Network and Node classes, and helper functions
from library import *

In [ ]:
def read_serial(device):
    """
    Reads the SRAM contents of a device

    Parameters
    ----------
    device: Node 
        Node class object
    Returns
    -------
    None
    """
    trace = []
    try:
        for i in range(32):
            line = device.serial.readline().decode('utf-8')[:-2]
            if line[0] == '\r':
                for i in line[1:].split(' '):
                    trace.append(int(i,16)) 
            else:
                for i in line.split(' '):
                    trace.append(int(i,16))
    
    except Exception as error:
        print('Error: ', error)
    
    device.memory.append(trace)

def collect_data(network,num_samples,filename,delay):
    """
    Helper function to read data from a Network.

    Parameters
    ----------
    network: Network 
        Network class object
    num_samples: int
        Number of samples to be collected.
    filename: str
        Name of the storage file.
    delay: int
        Attestation duration.
    Returns
    -------
    None
    """ 
    network.start()

    # wait for the nodes to be ready
    time.sleep(2)
    for i in tqdm(range(num_samples)):
        network.nodes[0].serial.write('r'.encode())
        for i in network.nodes:
            read_serial(i)
        # change duration if necessary
        time.sleep(delay)

    network.stop()

    network.memory_to_array()
    network.get_golden_means()
    
    with open(f'./data/{filename}.pkl', 'wb') as outp: # change filename
        pickle.dump(network, outp, pickle.HIGHEST_PROTOCOL)

# Safe Networks

Data on safe behavior is collected from two overall types of networks:
1. `D1`,`D2`,`D3`,`D4` are `development datasets` that are colelcted from the same `development network` at different initializations. They are used for the following:
    1. Analyse normal behavior
    2. Train corresponding ML models
    3. Test the performance on different initializations on the same device

Development data

In [ ]:
D = Network('D4', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,438,490,394,430,446])
collect_data(network=D, num_samples=1000, filename=D.name, delay=0.1)

# Anomalous Networks

The following node-level anomalies are introduced at one or more nodes in the network to create nine anomalous scenarios:

`N0`: N0 sends an attestation signal to N1-N5. However, it has some added functionality.\
`N1`: N1 generates random flating point in a different range than normal behavior and sends them to N2.\
`N2`: N2 receives valid sense values from N1 but does not save/use them. Instead, it generates random control signals for N3.\
`N3`: N3 receives valid control signals from N2, but does not save/use them. Instead, it generates random output signals for its peripherals.\
`N4`: N4 generates correct data but does not send anything to N5.\
`N5`: N5 controls a different set of output devices.

`AN0`: Anomaly at N0

In [ ]:
A = Network('AN0', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [199,438,490,394,430,446])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

`AN1`: Anomaly at N1

In [ ]:
A = Network('AN1', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,430,490,394,430,446])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

`AN2`: Anomaly at N2

In [ ]:
A = Network('AN2', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,438,414,394,430,446])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

`AN3`: Anomaly at N3

In [ ]:
A = Network('AN3', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,438,490,386,430,446])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

`AN4`: Anomaly at N4

In [ ]:
A = Network('AN4', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,438,490,394,372,446])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

`AN5`: Anomaly at N5

In [ ]:
A = Network('AN5', ['N0','N1','N2','N3','N4','N5'], ['COM3','COM9','COM10','COM11','COM13','COM12'], [195,438,490,394,430,452])
collect_data(network=A, num_samples=1000, filename=A.name, delay=0.1)

# Compiled Data

In [ ]:
networks = ['D1','D2','D3','D4','AN0','AN1','AN2','AN3','AN4','AN5'] # 10 scenarios
swarm_attestation_dataset = {}
for name in tqdm(networks):
    with open(f'./raw-data/swarm-2/{name}.pkl','rb') as inp:
        network = pickle.load(inp)
        swarm_attestation_dataset[network.name] = {node.name:{'SRAM':node.memory[100:],'threshold':node.threshold} for node in network.nodes}
    del network

with open('/data/swarm-2.pkl','wb') as outp:
    pickle.dump(swarm_attestation_dataset, outp, pickle.HIGHEST_PROTOCOL)
del outp